### Demo RAG with: Oracle AI Vector Store, OCI GenAI Embeddings, OCI GenAI Llama2 and LangChain

In [1]:
import logging

import oci
import ads
import oracledb

from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from oci_utils import load_oci_config, pretty_print_docs, format_docs
from ads.llm import GenerativeAIEmbeddings, GenerativeAI

# regarding sdk we have two dependencies here:
# oracledb for the Vector Store
# oci sdk for supporting ads that gives the GenerativeAIEmbeddings
# eventually, the embedding model can be changed

# my OracleVectorStore for LangChain
from oracle_vector_db_lc import OracleVectorStore

# EMBED_MODEL = "cohere.embed-multilingual-v3.0"
from config import EMBED_MODEL

# you need to have a config_private.py file where you specify COMPARTMENT_OCID, ENDPOINT
# ENDPOINT = "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"
from config_private import COMPARTMENT_OCID, ENDPOINT

#### Setup
* security
* clients (Embeddings, LLM, Vector Store)

In [2]:
# num. of documents retrieved from the Vector Store

TOP_K = 4

# security (pass the api keys)
oci_config = load_oci_config()

api_keys_config = ads.auth.api_keys(oci_config)

# the default prompt we're using
prompt = hub.pull("rlm/rag-prompt")

#
# clients to the Embeddings and LLM models
#
embed_model = GenerativeAIEmbeddings(
    compartment_id=COMPARTMENT_OCID,
    model=EMBED_MODEL,
    auth=api_keys_config,
    # here we specify the endpoint for GenAI
    client_kwargs={"service_endpoint": ENDPOINT},
)

#
# Choice of the LLM for response generation
#
GEN_MODEL = "LLAMA2"

if GEN_MODEL == "COHERE":
    llm = GenerativeAI(
        auth=api_keys_config,
        compartment_id=COMPARTMENT_OCID,
        max_tokens=1024,
        temperature=0.1,
        verbose=False,
        client_kwargs={"service_endpoint": ENDPOINT},
    )

if GEN_MODEL == "LLAMA2":
    llm = GenerativeAI(
        auth=api_keys_config,
        compartment_id=COMPARTMENT_OCID,
        model="meta.llama-2-70b-chat",
        max_tokens=1024,
        temperature=0.1,
        verbose=False,
        client_kwargs={"service_endpoint": ENDPOINT},
    )

In [3]:
# Initialising in this way we link with the embeddings model
# we have used OCI GenAI based on Cohere but you could plug any other model
# need to pass only the embedding functions

v_store = OracleVectorStore(embedding_function=embed_model.embed_query, verbose=True)

#### Similarity Search on Vector Store

In [4]:
question = "What is JSON Relational Duality?"
# question = "What are the symptoms of long covid fro adults? provide details."

In [5]:
docs = v_store.similarity_search(query=question, k=TOP_K)

pretty_print_docs(docs)

2024-01-20 12:32:33,993 - INFO - top_k: 4
2024-01-20 12:32:33,993 - INFO - 
2024-01-20 12:32:35,162 - INFO - select: select V.id, C.CHUNK, C.PAGE_NUM, 
                            ROUND(VECTOR_DISTANCE(V.VEC, :1, DOT), 3) as d,
                            B.NAME 
                            from VECTORS V, CHUNKS C, BOOKS B
                            where C.ID = V.ID and
                            C.BOOK_ID = B.ID
                            order by d
                            FETCH FIRST 4 ROWS ONLY
2024-01-20 12:32:35,433 - INFO - Query duration: 0.5 sec.


Document 1:

2 Application Development JSON JSON-Relational Duality JSON Relational Duality Views are fully updatable JSON views over relational data. Data is still stored in relational tables in a highly efficient normalized format but can be accessed by applications in the form of JSON documents. Duality views provide you with game-changing flexibility and simplicity by overcoming the historical challenges developers have faced when building applications using relational or document models. Related Resources View Documentation JSON Schema JSON Schema-based validation is allowed with the SQL condition IS JSON and with a PL/SQL utility function. A JSON schema is a JSON document that specifies allowed properties (field names) and the corresponding allowed data types, and whether they are optional or mandatory. By default, JSON data is schemaless, providing flexibility. However, you may want to ensure that your JSON data contains particular mandatory fixed structures and typing, besides 

#### The same search using a Retriever
* in a retriever you could plug post processing; for example: reranking

In [6]:
# This is the way to specify top_k
retriever = v_store.as_retriever(search_kwargs={"k": TOP_K})

In [7]:
docs = retriever.get_relevant_documents(question)

2024-01-20 12:32:35,463 - INFO - top_k: 4
2024-01-20 12:32:35,464 - INFO - 
2024-01-20 12:32:35,917 - INFO - select: select V.id, C.CHUNK, C.PAGE_NUM, 
                            ROUND(VECTOR_DISTANCE(V.VEC, :1, DOT), 3) as d,
                            B.NAME 
                            from VECTORS V, CHUNKS C, BOOKS B
                            where C.ID = V.ID and
                            C.BOOK_ID = B.ID
                            order by d
                            FETCH FIRST 4 ROWS ONLY
2024-01-20 12:32:36,188 - INFO - Query duration: 0.4 sec.


In [8]:
pretty_print_docs(docs)

Document 1:

2 Application Development JSON JSON-Relational Duality JSON Relational Duality Views are fully updatable JSON views over relational data. Data is still stored in relational tables in a highly efficient normalized format but can be accessed by applications in the form of JSON documents. Duality views provide you with game-changing flexibility and simplicity by overcoming the historical challenges developers have faced when building applications using relational or document models. Related Resources View Documentation JSON Schema JSON Schema-based validation is allowed with the SQL condition IS JSON and with a PL/SQL utility function. A JSON schema is a JSON document that specifies allowed properties (field names) and the corresponding allowed data types, and whether they are optional or mandatory. By default, JSON data is schemaless, providing flexibility. However, you may want to ensure that your JSON data contains particular mandatory fixed structures and typing, besides 

#### Build an entire RAG chain and answer to the question using a LLM

In [9]:
# using LangChain LCEL language
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

##### Answer to the question

In [11]:
%%time
answer = rag_chain.invoke(question)

print(f"Question: {question}")
print("")
print(answer)
print("")

2024-01-20 12:35:17,379 - INFO - top_k: 4
2024-01-20 12:35:17,383 - INFO - 
2024-01-20 12:35:17,912 - INFO - select: select V.id, C.CHUNK, C.PAGE_NUM, 
                            ROUND(VECTOR_DISTANCE(V.VEC, :1, DOT), 3) as d,
                            B.NAME 
                            from VECTORS V, CHUNKS C, BOOKS B
                            where C.ID = V.ID and
                            C.BOOK_ID = B.ID
                            order by d
                            FETCH FIRST 4 ROWS ONLY
2024-01-20 12:35:18,175 - INFO - Query duration: 0.4 sec.


Question: What is JSON Relational Duality?

 Here are 5 new features for JSON in Oracle Database 23c:

1. JSON-Relational Duality: Developers can access and update JSON data as either JSON documents or relational tables, benefiting from the strengths of both.
2. Operational Property Graphs in SQL: Developers can build real-time graph analysis applications against operational data directly in the Oracle Database, utilizing its industry-leading security, high availability, and performance capabilities.
3. Microservice Support: Oracle Database 23c provides simplified cross-service transactions, making it easier to implement microservices.
4. Lock-Free Column Value Reservations: Applications can reserve part of a value in a column without locking the row, allowing for more efficient operations on bank accounts, inventory, and other data.
5. Native Database Support for JSON: Oracle Database supports JSON natively with features such as transactions, indexing, declarative querying, and views,